In [1]:
import tensorflow as tf
import numpy as np

#### 1.假设我们输入的矩阵为[5, 5, 3]

In [3]:
import tensorflow as tf
import numpy as np

# 构建[5, 5, 3]的矩阵
M = np.random.randint(-10, 10, size=[5, 5, 3])

# 打印矩阵的数据
print("Matrix shape is: ", M.shape)
print("Matrix data is: ", M.data)

Matrix shape is:  (5, 5, 3)
Matrix data is:  <memory at 0x000001CDFFCD9D68>


#### 2.定义卷积过滤器，为1@[5, 5, 3]

In [4]:
# 通过tf.get_variable()函数创建过滤器
# 这里申明的参数为4维变量，
# 前两个参数表示过滤尺寸
# 第三个参数表示输入层的深度
# 第四个参数表示过滤器的个数
filter_weight = tf.get_variable("weights",
                               shape=[5, 5, 3, 1],
                               initializer=tf.constant_initializer(1.0))
# 设置偏置项
biases = tf.get_variable("biases", shape=[1], 
                         initializer=tf.constant_initializer(0.1))

#### 3.调整输入的格式符合TensorFlow的要求

In [5]:
M = np.asarray(M, dtype='float32') # 调整类型
M = M.reshape(1, 5, 5, 3)

#### 4.计算矩阵通过卷积层过滤和池化层过滤器计算后的结果

In [6]:
# 构建输入节点
# [1, None, None, None]:
# 第一个维度：表示batch的大小
# 后面三个维度：表示图片尺寸的大小
x = tf.placeholder(tf.float32, [1, None, None, None])

# 构建卷积层
# tf.nn.conv2d提供了一个非常方便的函数来实现卷积层前向传播算法。
# 第一个参数：为当前层的节点矩阵。注意这个节点矩阵是一个四维矩阵，
# 后面三维对应一个节点矩阵，第一维对应一个输入batch
# 比如：input[0, :, :, :]表示第一张图片；
# input[1, :, :, :]表示第二张图片；后续以此类推
# 第二个参数提供了卷积层的权重，
# 第三个参数为不同维度上的步长,虽然第三个参数提供的是一个长度为4的数组，
# 但是第一维和最后一维的数字一定是1.这是因为卷积的步长只对矩阵的长宽有效
# 最后一个参数是填充(padding)的方法，TensorFlow中提供SAME或者VALID两种选择。
# 其中SAME表示添加全0填充，VALID表示不添加
conv = tf.nn.conv2d(x, filter_weight, strides=[1, 2, 2, 1], padding='SAME')

# tf.nn.bias_add提供了一个方便的函数给每一个节点加上偏置项。
# 注意这里不能直接使用加法
# 因为矩阵上不同位置的节点上都需要加上同样的偏置项
layer = tf.nn.bias_add(conv, biases)

# 构建池化层
# tf.nn.max_pool实现了最大池化层的前向传播。
# ksize：提供了过滤器的尺寸，四维矩阵，第一个参数与最后一个参数必须为1
# strides：提供了步长信息，四维矩阵，第一个参数和最后一个参数必须为1，
# 常见池化层过滤器尺寸为[1,2,2,1]或者[1,3,3,1]
# padding：提供了是否使用全零填充
# tf.nn.avg_pool实现了平均池化。参数与tf.nn.max_pool()函数一样
pool = tf.nn.max_pool(x, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

# 建立会话
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allow_growth=True
with tf.Session(config=config) as sess:
    # 变量初始化
    tf.global_variables_initializer().run()
    
    # 计算卷积层
    convoluted_M = sess.run(layer, feed_dict={x:M})
    
    # 计算池化层
    pooled_M = sess.run(pool, feed_dict={x:M})
    
    # 打印结果
    print("convoluted_M: \n", convoluted_M)
    print("pooled_M: \n", pooled_M)

convoluted_M: 
 [[[[  6.0999999 ]
   [ -7.9000001 ]
   [ -9.89999962]]

  [[-36.90000153]
   [-21.89999962]
   [-12.89999962]]

  [[-29.89999962]
   [-30.89999962]
   [-24.89999962]]]]
pooled_M: 
 [[[[ 7.  1.  4.]
   [ 9.  7.  7.]
   [-1.  5.  3.]]

  [[ 9.  7.  8.]
   [ 5.  7.  9.]
   [ 5.  8. -3.]]

  [[ 0.  8.  9.]
   [-2.  8. -7.]
   [ 6.  9. -1.]]]]
